In [1]:
batch_size = 64

In [2]:
import torch
from src.models.elg import ELG
elg_model = ELG().cuda()

In [3]:
from src.datasources.unityeyes import UnityEyesDataset
train_root = "E:/Datasets/UnityEyes_Windows/800x600/train"
train_dataset = UnityEyesDataset(train_root, eye_image_shape=(36, 60), random_difficulty=True)
val_root = "E:/Datasets/UnityEyes_Windows/800x600/val"
val_dataset = UnityEyesDataset(val_root, eye_image_shape=(36, 60), random_difficulty=True)

In [4]:
import torch.utils.data as D
train_dataloader = D.DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = D.DataLoader(val_dataset, batch_size=batch_size)

In [5]:
from src.trainers.elg_trainer import ELGTrainer
elg_trainer = ELGTrainer(model=elg_model, epochs=10, batch_size=batch_size, version='v0.0')

In [ ]:
elg_trainer.run(train_dataloader, val_dataloader)

Start epoch 1 with learning rate 0.0001
Start training...


In [9]:
one_batch = next(iter(train_dataloader))
one_batch

{'radius': tensor([32.5454, 30.3245, 38.7351, 39.3610, 37.2926, 35.8716, 25.7081, 40.1915,
         37.4513, 31.8688, 31.5459, 37.0810, 30.3742, 35.5046, 31.3968, 30.4410,
         34.0830, 33.0046, 33.3169, 43.5449, 35.7848, 34.9777, 29.5343, 37.6764,
         29.6483, 33.9501, 26.8504, 30.4112, 44.4663, 30.6892, 42.2581, 35.6033]),
 'gaze': tensor([[-0.3596,  0.3704],
         [-0.5455,  0.5129],
         [ 0.1662, -0.0862],
         [ 0.3635,  0.0597],
         [-0.0346, -0.5882],
         [-0.3840, -0.5500],
         [ 0.1006, -0.0726],
         [ 0.1733,  0.1335],
         [ 0.0835,  0.2047],
         [-0.4137, -0.6612],
         [-0.4326, -0.2134],
         [-0.3367,  0.4689],
         [-0.0109, -0.0901],
         [-0.0594,  0.3431],
         [-0.2504,  0.1735],
         [ 0.1910, -0.2878],
         [ 0.2817,  0.1470],
         [ 0.3537, -0.4985],
         [-0.4646,  0.6708],
         [-0.0846, -0.0136],
         [-0.1933, -0.1130],
         [-0.4479, -0.0659],
         [ 0.3084,

In [40]:
import torch
import math
def compute_angular_loss(predict, label):
    """Pytorch method to calculate angular loss (via cosine similarity)"""
    def angle_to_unit_vectors(y):
        sin = torch.sin(y)
        cos = torch.cos(y)
        return torch.stack([
            cos[:, 0] * sin[:, 1],
            sin[:, 0],
            cos[:, 0] * cos[:, 1],
        ], dim=1)

    a = angle_to_unit_vectors(predict)
    b = angle_to_unit_vectors(label)
    ab = torch.sum(a*b, dim=1)
    a_norm = torch.sqrt(torch.sum(torch.square(a), dim=1))
    b_norm = torch.sqrt(torch.sum(torch.square(b), dim=1))
    cos_sim = ab / (a_norm * b_norm)
    cos_sim = torch.clip(cos_sim, -1.0 + 1e-6, 1.0 - 1e-6)
    ang = torch.acos(cos_sim) * 180. / math.pi
    return torch.mean(ang)

In [41]:
a = torch.tensor([[2.0, 2.0], [2.0, 2.0]])
b = torch.tensor([[2.0, 2.0], [2.0, 2.0]])

In [42]:
a

tensor([[2., 2.],
        [2., 2.]])

In [43]:
b

tensor([[2., 2.],
        [2., 2.]])

In [44]:
compute_angular_loss(a, b)

tensor(0.0816)

In [45]:
torch.acos(torch.tensor(1.0))

tensor(0.)

In [1]:
import gaze_train

In [2]:
gaze_train.train(64, (36, 60), 100, version=f'v0.2-Gaze')

Start epoch 1 with learning rate 0.0001
Start training...
Trained batch: 500.0 Batch loss: 20.69852638244629 Epoch total loss: 10556.35735321045


KeyboardInterrupt: 